# Testing EfficientNet Locally Before Transferring To The Cloud

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

from src.data.prepare_data import *
from src.models.model import *
import time
import random
from sklearn.model_selection import KFold
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data.sampler import WeightedRandomSampler
from tqdm import tqdm

In [3]:
random.seed(2718)
np.random.seed(2718)
torch.manual_seed(2718)
SEED = 2718

In [4]:
train = pd.read_csv("../data/internal/train.csv")
test = pd.read_csv("../data/internal/test.csv")
sub = pd.read_csv("../data/internal/sample_submission.csv")

This model has enough parameters that it is not feasible to train a model with larger than 64x64 images.

In [5]:
config = {
    'INPUT_DIR'      : '',
    'MODEL'          : 'efficientnet-b0',
    'SIZE'           : 128,
    'BATCH_SIZE'     : 32,
    'NUM_FOLDS'      : 3,
    'NUM_EPOCHS'     : 20,
    'FREEZED_EPOCHS' : 3,
    'LEARNING_RATE'  : 1e-3,
    'EARLY_STOPPING' : 10,
    'UNIFORM_AUGMENT': True,
    'TTA'            : 3,
    'NUM_WORKERS'    : 16,
    'DEVICE'         : 'cpu'
}

In [6]:
t = time.time()
predictions = pd.DataFrame()
transform = ImageTransform(config['SIZE'], config['UNIFORM_AUGMENT'])

skf = KFold(n_splits=config['NUM_FOLDS'], shuffle=True, random_state=SEED)
for i, (idxT,idxV) in enumerate(skf.split(np.arange(15))):
    if i == 2:
        t_fold = time.time()
        tr = train.loc[train.tfrecord.isin(idxT)]
        va = train.loc[train.tfrecord.isin(idxV)]
        tr.reset_index(drop=True, inplace=True)
        va.reset_index(drop=True, inplace=True)

        # create datasets
        dataset_train = MelanomaDataset("../data/internal/train", tr, transform=transform, phase='train')
        dataset_valid = MelanomaDataset("../data/internal/train", va, transform=transform, phase='valid')

        # load a pretrained model
        net = load_model(config['MODEL'], 2)

        # define a loss function
        criterion = nn.CrossEntropyLoss()

        # define an optimizer
        optimizer = optim.Adam(net.parameters(), lr=config['LEARNING_RATE'])

        # define a scheduler
        scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=2, factor=0.2)

        # create a sampler
        sampler = create_weighted_random_sampler(tr)

        # train the network
        print(f"---- fold: {i + 1} ------------")
        train_model(
            f"{config['MODEL']}_{i + 1}",
            dataset_train,
            dataset_valid,
            config['BATCH_SIZE'],
            net,
            criterion,
            optimizer,
            scheduler,
            config['NUM_EPOCHS'],
            config['FREEZED_EPOCHS'],
            config['INPUT_DIR'],
            config['NUM_WORKERS'],
            sampler,
            config['DEVICE'],
            config['EARLY_STOPPING']
        )

        # predict on test dataset
        test['target'] = 0
        dataset_test = MelanomaDataset("../data/internal/test", test, transform=transform, phase='test')
        predictions = get_predictions(dataset_test, 
                                      config["BATCH_SIZE"], 
                                      net, 
                                      config["TTA"], 
                                      predictions, 
                                      config["DEVICE"])
        predictions.to_csv(f'../submissions/{config["MODEL"]}_fold{i+1}.csv')
        print(f"fold took {round(time.time() - t_fold, 2)}")
    
# output
sub['target'] = predictions.mean(axis=1)
sub.to_csv(f"../submissions/submission{config['MODEL']}.csv", index=False)
print(f"total time: {round(time.time() - t, 4)}")

  0%|          | 0/682 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b0
---- fold: 3 ------------


  0%|          | 0/682 [00:00<?, ?it/s]

epoch: 1, loss_train: 0.5630, loss_valid: 0.3856, auc_valid: 0.8116, saved: True, 750.1975sec


  0%|          | 0/682 [00:00<?, ?it/s]

...

epoch: 14, loss_train: 0.1078, loss_valid: 0.1085, auc_valid: 0.8658, saved: False, 1707.2401sec


  0%|          | 0/3 [00:00<?, ?it/s]

epoch: 15, loss_train: 0.1033, loss_valid: 0.1081, auc_valid: 0.8660, saved: False, 1717.6983sec


100%|██████████| 3/3 [09:42<00:00, 194.06s/it]

fold took 24838.69
total time: 24838.7177
